In [23]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [24]:
import tensorflow as tf
import numpy as np

tf.enable_eager_execution()

In [44]:
X = np.load("/data/elsa/elsa_ja_X.npy", allow_pickle=True).astype(np.int32)
y = np.load("/data/elsa/elsa_ja_y.npy", allow_pickle=True).astype(np.int32)

In [45]:
train_dataset = tf.data.Dataset.from_tensor_slices((X, y))

In [46]:
train_dataset = train_dataset.batch(32)
train_dataset

<DatasetV1Adapter shapes: ((?, 50), (?, 64)), types: (tf.int32, tf.int32)>

In [47]:
train_dataset.output_types

(tf.int32, tf.int32)

In [48]:
iterator = train_dataset.make_one_shot_iterator()
iterator

In [49]:
next_data = iterator.get_next()

In [51]:
next_data[0].shape

TensorShape([Dimension(32), Dimension(50)])

In [52]:
with tf.python_io.TFRecordWriter("/data/elsa/test.tfrecords") as writer:
    for idx in range(len(X)):
        feature = {
            "X": tf.train.Feature(int64_list=tf.train.Int64List(value=X[idx])),
            "Y": tf.train.Feature(int64_list=tf.train.Int64List(value=y[idx]))
        }
        features = tf.train.Features(feature=feature)
        example = tf.train.Example(features=features)
        serialized = example.SerializeToString()
        writer.write(serialized)

In [27]:
for serialized_example in tf.python_io.tf_record_iterator('/data/elsa/test.tfrecords'):
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    x_1 = np.array(example.features.feature['X'].int64_list.value)
    y_1 = np.array(example.features.feature['Y'].int64_list.value)
    print(example)
    break

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


142

features {
  feature {
    key: "X"
    value {
      int64_list {
        value: 191
        value: 68
        value: 30
        value: 32
        value: 74
        value: 77
        value: 254
        value: 237
        value: 75
        value: 24
        value: 62
        value: 36
        value: 34
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
      }
    }
  }
  feature {
    key: "Y"
    value {
      int64_

In [53]:
raw_dataset = tf.data.TFRecordDataset(['/data/elsa/test.tfrecords'])
raw_dataset

<TFRecordDatasetV1 shapes: (), types: tf.string>

In [54]:
for record in raw_dataset.take(3):
    print(repr(record))

<tf.Tensor: id=164, shape=(), dtype=string, numpy=b'\n\x8b\x01\n>\n\x01X\x129\x1a7\n5\xbf\x01D\x1e JM\xfe\x01\xed\x01K\x18>$"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\nI\n\x01Y\x12D\x1aB\n@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'>
<tf.Tensor: id=166, shape=(), dtype=string, numpy=b'\n\x8e\x01\nA\n\x01X\x12<\x1a:\n8\xbe\x01\n\xc0\x06\x9c\x1c\xf0\x17\x98$N\xe3\x0b5\x16\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\nI\n\x01Y\x12D\x1aB\n@\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

In [55]:
feature_description = {
    'X': tf.FixedLenFeature([], tf.int64, default_value=0),
    'Y': tf.FixedLenFeature([], tf.int64, default_value=0),
}

def _parse_function(example_proto):
    features = tf.parse_single_example(example_proto, feature_description)
    X = tf.cast(features['X'], tf.int32)
    Y = tf.cast(features['Y'], tf.int32)
    return X, Y

In [56]:
parsed_dataset = raw_dataset.map(_parse_function)
parsed_dataset 

<DatasetV1Adapter shapes: ((), ()), types: (tf.int32, tf.int32)>

In [58]:
iterator = parsed_dataset.make_one_shot_iterator()
data = iterator.next()

InvalidArgumentError: Key: Y.  Can't parse serialized Example.
	 [[{{node ParseSingleExample/ParseSingleExample}}]] [Op:IteratorGetNextSync]

In [59]:
for record in parsed_dataset.take(3):
    print(record)

InvalidArgumentError: Key: Y.  Can't parse serialized Example.
	 [[{{node ParseSingleExample/ParseSingleExample}}]] [Op:IteratorGetNextSync]